In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
# for ensemble method use
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import chi2
from imblearn.over_sampling import SMOTE
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


#TO DO-
#One-R
# sampling SOMTE


# height removed
Test_Target = ["SUSPECT_ARRESTED_FLAG", "STOP_WAS_INITIATED",
                 "OBSERVED_DURATION_MINUTES", "SUSPECTED_CRIME_DESCRIPTION",
                 "STOP_DURATION_MINUTES", "FIREARM_FLAG",
                 "SUSPECT_RACE_DESCRIPTION",
                 "DEMEANOR_OF_PERSON_STOPPED", "SUSPECT_BODY_BUILD_TYPE","FRISKED_FLAG","SEARCHED_FLAG","STOP_LOCATION_BORO_NAME"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)



In [ ]:

nlpp = pd.read_excel("sqf2022.xlsx", usecols=["DEMEANOR_OF_PERSON_STOPPED"])
print(len(nlpp))
nlp = nlpp.fillna(value='neutral')
print(len(nlp))

# Assuming df is your DataFrame and 'column_name' is the name of the column you want to check
null_count = nlp['DEMEANOR_OF_PERSON_STOPPED'].isnull().sum()
print(len(nlp))
print(f"Number of null values in 'column_name': {null_count}")


In [ ]:
dataa = nlp['DEMEANOR_OF_PERSON_STOPPED'].tolist()
analyzer = SentimentIntensityAnalyzer()
new_data = [ analyzer.polarity_scores(element)['compound'] for element in dataa]

In [ ]:
new=[]
for pair in data:
    new.append(pair)
# Create DataFrame from list of tuples
final_df = pd.DataFrame(new, columns=['DEMEANOR_OF_PERSON_STOPPED', 'COMPOUND_SENTIMENT'])

In [ ]:
Feature_test=["FIREARM_FLAG","FRISKED_FLAG","SEARCHED_FLAG"]

Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', np.nan, inplace=True)

lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])
Test1["FRISKED_FLAG"] = lb.fit_transform(Test1["FRISKED_FLAG"])
Test1["SEARCHED_FLAG"] = lb.fit_transform(Test1["SEARCHED_FLAG"])


Test1["SUSPECT_ARRESTED_FLAG"].fillna(Test1["SUSPECT_ARRESTED_FLAG"].median,inplace=True)
Test1["FIREARM_FLAG"].fillna(Test1["FIREARM_FLAG"].median,inplace=True)
Test1["FRISKED_FLAG"].fillna(Test1["FRISKED_FLAG"].median,inplace=True)
Test1["SEARCHED_FLAG"].fillna(Test1["SEARCHED_FLAG"].median,inplace=True)

In [56]:
#One Hot-Encoding
OneHot_Encode_feature = ["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION"]
Label_En = OneHotEncoder(handle_unknown="ignore")
Feature_Encoded = Label_En.fit_transform(Test1[OneHot_Encode_feature])
Label_df = pd.DataFrame(Feature_Encoded.toarray(), columns=Label_En.get_feature_names_out(OneHot_Encode_feature))


In [57]:
#label encoding
LE=LabelEncoder()
Test1['SUSPECT_BODY_BUILD_TYPE']=LE.fit_transform(Test1["SUSPECT_BODY_BUILD_TYPE"])
Test1['SUSPECT_BODY_BUILD_TYPE'].fillna(Test1['SUSPECT_BODY_BUILD_TYPE'].mean, inplace=True)

In [58]:
# from sklearn.feature_extraction.text import CountVectorizer
# import numpy as np

# # Assuming you have a list of texts in the 'DEMEANOR' feature
# texts = Test1['DEMEANOR_OF_PERSON_STOPPED'].fillna('').tolist()

# # Initialize the CountVectorizer
# vectorizer = CountVectorizer()

# # Fit and transform the texts using the CountVectorizer
# X = vectorizer.fit_transform(texts)

# # Get the feature names
# feature_names = vectorizer.get_feature_names()

# # Convert the vectorized data to a DataFrame
# vectorized_data = pd.DataFrame(X.toarray(), columns=feature_names)

# # Print the vectorized data
# print(vectorized_data)


In [59]:
nlpp = pd.read_excel("sqf2022.xlsx", usecols=["DEMEANOR_OF_PERSON_STOPPED"])
print(len(nlpp))
nlp = nlpp.fillna(value='neutral')
print(len(nlp))

# Assuming df is your DataFrame and 'column_name' is the name of the column you want to check
null_count = nlp['DEMEANOR_OF_PERSON_STOPPED'].isnull().sum()
print(len(nlp))
print(f"Number of null values in 'DEMEANOR_OF_PERSON_STOPPED': {null_count}")
data = nlp['DEMEANOR_OF_PERSON_STOPPED'].tolist()
analyzer = SentimentIntensityAnalyzer()
new_data = [analyzer.polarity_scores(element)['compound'] for element in data]
new = [(pair, compound) for pair, compound in zip(data, new_data)]
# Create DataFrame from list of tuples
final_df = pd.DataFrame(new, columns=['DEMEANOR_OF_PERSON_STOPPED', 'COMPOUND_SENTIMENT'])


15102
15102
15102
Number of null values in 'DEMEANOR_OF_PERSON_STOPPED': 0


In [60]:

Test1 = pd.concat([Test1, Label_df], axis=1)
#Test1=pd.concat([Test1,vectorized_data],axis=1)
Test1 = pd.concat([Test1, final_df], axis=1)

#cat_feature=list(Label_df.columns)+Feature_test+list(vectorized_data.columns)
cat_feature=list(Label_df.columns)+Feature_test+["COMPOUND_SENTIMENT"]

# drop the original categorical feature column
Test1 = Test1.drop(columns=OneHot_Encode_feature)
Test1= Test1.drop(columns="DEMEANOR_OF_PERSON_STOPPED")

Test1.to_csv('output.csv', index=False)





train_data,test_data=train_test_split(Test1,test_size=0.2,random_state=42)


#classi
X_train = train_data[cat_feature] # training features_for Random forest
y_train = train_data['SUSPECT_ARRESTED_FLAG'] # training target variable for random forest

X_test = test_data[cat_feature] # testing features
y_test = test_data['SUSPECT_ARRESTED_FLAG'] # testing target variable


smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

In [61]:
# Model_N=MultinomialNB()
# Model_N.fit(X_train,y_train)
# N_prediction=Model_N.predict(X_test)
# train_N_prediction=Model_N.predict(X_train)
# N_train_accuracy=accuracy_score(y_train,train_N_prediction)
# N_accuracy=accuracy_score(y_test,N_prediction)

# print('Train_Accuracy:', N_accuracy)
# print('Accuracy:', N_train_accuracy)


In [62]:
# Model_N_Over = MultinomialNB()
# Model_N_Over.fit(X_train_oversampled, y_train_oversampled)
# N_prediction_Over = Model_N_Over.predict(X_test)
# train_N_prediction_Over = Model_N_Over.predict(X_train_oversampled)
# N_train_accuracy_Over = accuracy_score(y_train_oversampled, train_N_prediction_Over)
# N_accuracy_Over = accuracy_score(y_test, N_prediction_Over)

# print('Train_Accuracy:', N_train_accuracy_Over)
# print('Accuracy:', N_accuracy_Over)

In [63]:
model_R = RandomForestClassifier(random_state=42)
#model_R.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])

#with only caterogical data but transform to numeric/ with both

model_R.fit(X_train,y_train)
Prediction=model_R.predict(X_test)

train_Prediction =model_R.predict(X_train)


train_accuracy=accuracy_score(y_train,train_Prediction)

accuracy = accuracy_score(y_test, Prediction)


print('Train_Accuracy:', train_accuracy)
print('Accuracy:', accuracy)


Train_Accuracy: 0.888585382004801
Accuracy: 0.8593181065872227


In [64]:
model_R_Over = RandomForestClassifier(random_state=42)
#model_R.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])

#with only caterogical data but transform to numeric/ with both

model_R_Over.fit(X_train_oversampled,y_train_oversampled)
Prediction_Over=model_R_Over.predict(X_test)

train_Prediction_Over =model_R_Over.predict(X_train_oversampled)


train_accuracy_Over=accuracy_score(y_train_oversampled,train_Prediction_Over)

accuracy_Over = accuracy_score(y_test, Prediction_Over)


print('Train_Accuracy:', train_accuracy_Over)
print('Accuracy:', accuracy_Over)


Train_Accuracy: 0.8811470333209463
Accuracy: 0.8497186362131744


In [65]:
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions on the testing and training data, and calculate RMSE
y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)

print('Testing RMSE:', test_rmse)
print('Training RMSE:', train_rmse)

Testing RMSE: 0.34011043399243346
Training RMSE: 0.3450124614237926


In [66]:
logreg = LogisticRegression(max_iter=1000,random_state=42)
logreg.fit(X_train, y_train)
log_predict=logreg.predict(X_test)
log_score=accuracy_score(y_test,log_predict)

train_log_predict=logreg.predict(X_train)
train_log_score=accuracy_score(y_train,train_log_predict)

print("Accuracy:",log_score)
print("Train Accuract:",train_log_score)

Accuracy: 0.8656074147633234
Train Accuract: 0.8627597053224071


In [67]:
logreg_Over = LogisticRegression(max_iter=1000, random_state=42)
logreg_Over.fit(X_train_oversampled, y_train_oversampled)
log_predict_Over = logreg_Over.predict(X_test)
log_score_Over = accuracy_score(y_test, log_predict_Over)

train_log_predict_Over = logreg_Over.predict(X_train_oversampled)
train_log_score_Over = accuracy_score(y_train_oversampled, train_log_predict_Over)

print("Accuracy:", log_score_Over)
print("Train Accuracy:", train_log_score_Over)


Accuracy: 0.8530287984111221
Train Accuracy: 0.8381023163631859


In [68]:
Dec_model=DecisionTreeClassifier(random_state=42)
Dec_model.fit(X_train,y_train)
Dec_predict=Dec_model.predict(X_test)
Train_Dec_predict=Dec_model.predict(X_train)
Dec_accuracy=accuracy_score(y_test,Dec_predict)
Train_Dec_accuracy=accuracy_score(y_train,Train_Dec_predict)
print("Decision Tree score: ",Dec_accuracy)
print("Train_Decision Tree score: ",Train_Dec_accuracy)


Decision Tree score:  0.8563389606090699
Train_Decision Tree score:  0.888585382004801


In [69]:
Dec_model_Over = DecisionTreeClassifier(random_state=42)
Dec_model_Over.fit(X_train_oversampled, y_train_oversampled)

# Make predictions on the testing data
Dec_predict_Over = Dec_model_Over.predict(X_test)

# Make predictions on the training data (oversampled)
Train_Dec_predict_Over = Dec_model_Over.predict(X_train_oversampled)

# Calculate accuracy scores
Dec_accuracy_Over = accuracy_score(y_test, Dec_predict_Over)
Train_Dec_accuracy_Over = accuracy_score(y_train_oversampled, Train_Dec_predict_Over)

print("Decision Tree score on testing data: ", Dec_accuracy_Over)
print("Decision Tree score on training data (oversampled): ", Train_Dec_accuracy_Over)


Decision Tree score on testing data:  0.8444223766964581
Decision Tree score on training data (oversampled):  0.8811470333209463


In [70]:
dummy_class = DummyClassifier(strategy="most_frequent")
dummy_class.fit(X_train,y_train)
y_predict=dummy_class.predict(X_test)
y_train_predict=dummy_class.predict(X_train)
Dummy_accuracy = accuracy_score(y_test, y_predict)
Dummy_train_accuracy = accuracy_score(y_train, y_train_predict)
print("Training error",Dummy_accuracy)
print("TestError:", Dummy_train_accuracy)


Training error 0.6765971532605097
TestError: 0.6682393841569406


In [71]:
Dec_model_OneR=DecisionTreeClassifier(random_state=42,max_depth=1)
Dec_model_OneR.fit(train_data["SEARCHED_FLAG"].values.reshape(-1, 1), train_data["SUSPECT_ARRESTED_FLAG"])
predictions = Dec_model_OneR.predict(test_data["SEARCHED_FLAG"].values.reshape(-1, 1))
accuracy_OneR = accuracy_score(test_data["SUSPECT_ARRESTED_FLAG"], predictions)
print("Accuracy:", accuracy_OneR)

Accuracy: 0.7871565706719629


In [72]:
estimators=[("logreg", logreg), ("rf", model_R), ("Dc", Dec_model)]#,("Naive",Model_N)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train, y_train)
#test our model on the test data
print("Voting Classifier",ensemble.score(X_test, y_test))

Voting Classifier 0.8619662363455809


In [73]:
G_Model=GradientBoostingClassifier(random_state=42) #look for the base alg !! Gradient Boosted Decision Trees?
G_Model.fit(X_train,y_train)
G_predict=G_Model.predict(X_test)
G_predict_train=G_Model.predict(X_train)
G_accracy=accuracy_score(y_test,G_predict)
G_accracy_train=accuracy_score(y_train,G_predict_train)
print("Gradient ", G_accracy)
print("Train Gradient ", G_accracy_train)

Gradient  0.8695796094008607
Train Gradient  0.8670639847694728


In [74]:
G_Model=GradientBoostingClassifier(random_state=42) #look for the base alg !! Gradient Boosted Decision Trees?
G_Model.fit(X_train_oversampled,y_train_oversampled)
G_predict=G_Model.predict(X_test)
G_predict_train=G_Model.predict(X_train_oversampled)
G_accracy=accuracy_score(y_test,G_predict)
G_accracy_train=accuracy_score(y_train_oversampled,G_predict_train)
print("Gradient ", G_accracy)
print("Train Gradient ", G_accracy_train)

Gradient  0.8563389606090699
Train Gradient  0.8457203022420414


In [75]:
models = [RandomForestClassifier(random_state=42),RandomForestClassifier(random_state=42),
            LogisticRegression(random_state=0,max_iter=1000),DecisionTreeClassifier(random_state=42)#,MultinomialNB()
         ]
for model in models:
    print(model, '\n')
    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
    print('Cross-Validation Accuracy:', score, '\n', '\n')


RandomForestClassifier(random_state=42) 

Cross-Validation Accuracy: 0.8523305874229724 
 

RandomForestClassifier(random_state=42) 

Cross-Validation Accuracy: 0.8523305874229724 
 

LogisticRegression(max_iter=1000, random_state=0) 

Cross-Validation Accuracy: 0.8616010488619519 
 

DecisionTreeClassifier(random_state=42) 

Cross-Validation Accuracy: 0.8516679932706245 
 

